In [ ]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time

Функция, которая ищет расстояние между 2 точками через Яндекс карты.
Она позволит рассчитать расстояние, которое нужно проехать машине с товаром от места производства сырья до склада и от склада до страны сбыта. Координаты склада и сырья беру как данность

In [3]:
def get_distance(lat1, lon1, lat2, lon2):
    driver = webdriver.Chrome()
    driver.get("https://yandex.ru/maps")
    try:
        my_search = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "input__control"))) #чекнуть, что класс, там еще bold, оно не оно
        my_search.send_keys(str(lat1), ' ', str(lon1))
        my_search.send_keys(Keys.RETURN) #имитирую нажание enter с клавиатуры
    
        #ввели координаты, теперь нужно, чтобы появилось окошко для вторых координат, хочу нажать на маршрутную кнопку
        my_path = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "action-button-view._type_route")))
        my_path.click()
        #теперь я оказалась в месте, где заполнено "куда" и хочу попасть в "откуда"
        
        #так же как заполняла search заполняю "откуда"
        time.sleep(2)
        froms= WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "input__control")))
        time.sleep(2)

        my_from = froms[1]
        my_from.send_keys(str(lat2), ' ', str(lon2))
        my_from.send_keys(Keys.RETURN) 
        length = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "auto-route-snippet-view__route-subtitle"))).text
        return length
    except:
        print('ошибка в момент обращения')
    finally:
        driver.quit()

#lat1, lon1 - координаты точки назначения
#lat2, lon2 - координаты точки отправления


In [4]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import pandas as pd

In [6]:
#подключаю датайрейм из первой части проекта
path = "C:/Users/User/Downloads/final_df.csv"
df = pd.read_csv(path)
#выкидываю, все, что не нужно
df = df[['Country name', 'latitude', 'longitude']]
print(df)

              Country name  latitude  longitude
0   Bosnia and Herzegovina  43.86700   18.41700
1                  Moldova  47.00000   28.91700
2               Kyrgyzstan  42.86700   74.60000
3               Montenegro  42.78300   19.46700
4                  Belarus  53.91700   27.55000
5                   Russia  55.75583   37.61722
6               Tajikistan  38.55000   68.80000
7                  Armenia  40.18300   44.51700
8                 Bulgaria  42.69750   23.32250
9               Azerbaijan  40.38300   49.83300
10                 Albania  41.31700   19.81700
11         North Macedonia  42.00000   21.43300
12            Turkmenistan  37.96700   58.33300
13                 Georgia  41.71700   44.78300
14                 Ukraine  50.45000   30.51700


In [7]:
#ищу расстояния между складом в казани и каждой из стран
df['Расстояние от склада до страны'] = None
lat_kaz = float(55.79083)
lon_kaz = float(49.11444) #координаты склада
for index, row in df.iterrows():
    lat_country = row['latitude']
    lon_country = row['longitude']
    factory_country = get_distance(lat_country, lon_country, lat_kaz, lon_kaz)
    print(factory_country)
    df.at[index, 'Расстояние от склада до страны'] = factory_country
print(df)

3 200 км
2 470 км
2 970 км
3 300 км
1 550 км
840 км
3 100 км
2 310 км
3 400 км
ошибка в момент обращения
None
3 600 км
3 400 км
2 980 км
2 040 км
2 270 км
              Country name  latitude  longitude Расстояние от склада до страны
0   Bosnia and Herzegovina  43.86700   18.41700                       3 200 км
1                  Moldova  47.00000   28.91700                       2 470 км
2               Kyrgyzstan  42.86700   74.60000                       2 970 км
3               Montenegro  42.78300   19.46700                       3 300 км
4                  Belarus  53.91700   27.55000                       1 550 км
5                   Russia  55.75583   37.61722                         840 км
6               Tajikistan  38.55000   68.80000                       3 100 км
7                  Armenia  40.18300   44.51700                       2 310 км
8                 Bulgaria  42.69750   23.32250                       3 400 км
9               Azerbaijan  40.38300   49.83300        

In [16]:
df_2 = df.dropna()
print(df_2)

              Country name  latitude  longitude Расстояние от склада до страны
0   Bosnia and Herzegovina  43.86700   18.41700                       3 200 км
1                  Moldova  47.00000   28.91700                       2 470 км
2               Kyrgyzstan  42.86700   74.60000                       2 970 км
3               Montenegro  42.78300   19.46700                       3 300 км
4                  Belarus  53.91700   27.55000                       1 550 км
5                   Russia  55.75583   37.61722                         840 км
6               Tajikistan  38.55000   68.80000                       3 100 км
7                  Armenia  40.18300   44.51700                       2 310 км
8                 Bulgaria  42.69750   23.32250                       3 400 км
10                 Albania  41.31700   19.81700                       3 600 км
11         North Macedonia  42.00000   21.43300                       3 400 км
12            Turkmenistan  37.96700   58.33300     

In [17]:
#теперь выбираю страны с минимальной дальностью доставки
df_sorted = df.sort_values(by='Расстояние от склада до страны')
df_top10 = df_sorted.head(10)
print(df_top10)

              Country name  latitude  longitude Расстояние от склада до страны
4                  Belarus    53.917     27.550                       1 550 км
13                 Georgia    41.717     44.783                       2 040 км
14                 Ukraine    50.450     30.517                       2 270 км
7                  Armenia    40.183     44.517                       2 310 км
1                  Moldova    47.000     28.917                       2 470 км
2               Kyrgyzstan    42.867     74.600                       2 970 км
12            Turkmenistan    37.967     58.333                       2 980 км
6               Tajikistan    38.550     68.800                       3 100 км
0   Bosnia and Herzegovina    43.867     18.417                       3 200 км
3               Montenegro    42.783     19.467                       3 300 км


In [18]:
#теперь ищу для каждого сорта растения расстояние от поля до склада, result_df - датасет с местами, где растет сырье и их координатами
path2 = "C:/Users/User/Downloads/result_df.csv"
result_df = pd.read_csv(path2)
result_df = result_df.drop(0)
print(result_df)


   Unnamed: 0            город   долгота    широта  растение цена
1           0  Санкт-Петербург  30.31667  59.95000   мелисса  100
2           1          Чусовой  57.81667  58.28333  иван-чай  250
3           2           Тюмень  65.53333  57.15000      мята  100


In [19]:
result_df['Расстояние от растения до склада'] = None
lat_kaz = float(55.79083)
lon_kaz = float(49.11444) #координаты склада
for index, row in result_df.iterrows():
    lat_plant = row['широта']
    lon_plant = row['долгота']
    factory_plant = get_distance(lat_kaz, lon_kaz, lat_plant, lon_plant)
    print(factory_plant) 
    result_df.at[index, 'Расстояние от растения до склада'] = factory_plant
print(result_df)

1 530 км
790 км
1 360 км
   Unnamed: 0            город   долгота    широта  растение цена  \
1           0  Санкт-Петербург  30.31667  59.95000   мелисса  100   
2           1          Чусовой  57.81667  58.28333  иван-чай  250   
3           2           Тюмень  65.53333  57.15000      мята  100   

  Расстояние от растения до склада  
1                         1 530 км  
2                           790 км  
3                         1 360 км  
